In [ ]:
import sys
import numpy             as np
import tensorflow        as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow            import random_uniform_initializer as rui

In [ ]:
from gudhi.representations import DiagramScaler, Padding, PerslayModel

# Input persistence diagram

In [ ]:
diag = [np.array([[0.,4.],[1.,2.],[3.,8.],[6.,8.]])]

In [ ]:
plt.scatter(diag[0][:,0], diag[0][:,1])
plt.plot([0.,6.],[0.,6.])
plt.show()

In [ ]:
diag = DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())]).fit_transform(diag)
diag = Padding(use=True).fit_transform(diag)

In [ ]:
plt.scatter(diag[0][:,0], diag[0][:,1])
plt.plot([0.,1.],[0.,1.])
plt.show()

In [ ]:
D = np.stack(np.array(diag, dtype=np.float32), 0)

In [ ]:
diagrams, empty_feats = [D], np.empty([1,0], dtype=np.float32)
perslayParameters = {}

# Persistence weight and permutation invariant operation

## Choose persistence weights

In [ ]:
perslayParameters["pweight_train"]  = False

In [ ]:
perslayParameters["pweight"]        = "power"
perslayParameters["pweight_power"]  = 2
perslayParameters["pweight_init"]   = 1.

In [ ]:
perslayParameters["pweight"]        = "grid"
perslayParameters["pweight_size"]   = [100,100]
perslayParameters["pweight_bnds"]   = ((-.001, 1.001), (-.001, 1.001))
perslayParameters["pweight_init"]   = np.tile(np.arange(0.,100.,1, dtype=np.float32)[np.newaxis,:], [100,1])

In [ ]:
perslayParameters["pweight"]        = "gmix"
perslayParameters["pweight_num"]    = 3
perslayParameters["pweight_init"]   = np.array(np.vstack([np.random.uniform(0.,1.,[2,3]), 
                                                          5.*np.ones([2,3])]), dtype=np.float32)

In [ ]:
perslayParameters["pweight"]        = None

## Choose permutation invariant operation

In [ ]:
perslayParameters["perm_op"] = "sum"

In [ ]:
perslayParameters["perm_op"] = "topk"
perslayParameters["keep"]    = 3

In [ ]:
perslayParameters["perm_op"] = "max"

In [ ]:
perslayParameters["perm_op"] = "mean"

# Persistence representation

## Persistence image

In [ ]:
perslayParameters["layer"]           = "Image"
perslayParameters["layer_train"]     = False
perslayParameters["image_size"]      = (100, 100)
perslayParameters["image_bnds"]      = ((-.501, 1.501), (-.501, 1.501))
perslayParameters["lvariance_init"]  = .1
perslayParameters["final_model"]     = tf.keras.Sequential([tf.keras.layers.Flatten()])

In [ ]:
model = PerslayModel(name="perslay", diagdim=2, perslay_parameters=[perslayParameters], rho="identity")
vector = model([diagrams, empty_feats])

In [ ]:
# Plot representation
V = np.flip(np.reshape(vector[0,:], [int(np.sqrt(vector[0,:].shape[0])), int(np.sqrt(vector[0,:].shape[0]))]), 0)
plt.figure()
plt.imshow(V, cmap="Purples")
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.show()
    
# Plot weight
if perslayParameters["pweight"] == "grid":
    W = model.vars[0][0].numpy()
    weights = np.flip(W, 0)
    plt.figure()
    plt.imshow(weights, cmap="Purples", zorder=1)
    ((xm,xM),(ym,yM)) = perslayParameters["pweight_bnds"]
    [xs, ys] = perslayParameters["pweight_size"]
    plt.scatter([int(xs*(x-xm)/(xM-xm)) for x in diag[0][:,0]], 
                [ys-int(ys*(y-ym)/(yM-ym)) for y in diag[0][:,1]], 
                s=10, color="red", zorder=2)
    plt.show()
        
if perslayParameters["pweight"] == "gmix":
    means = model.vars[0][0][:2,:].numpy()
    varis = model.vars[0][0][2:,:].numpy()
    x = np.arange(-.5, 1.5, .001)
    y = np.arange(-.5, 1.5, .001)
    xx, yy = np.meshgrid(x, y)
    z = np.zeros(xx.shape)
    for idx_g in range(means.shape[1]):
        z += np.exp(-((xx-means[0,idx_g])**2 * (varis[0,idx_g])**2 
                    + (yy-means[1,idx_g])**2 * (varis[1,idx_g])**2 ))
    plt.contourf(xx, yy, z)
    plt.scatter(diag[0][:,0], diag[0][:,1], s=50, color="red")
    plt.show()

## Persistence landscape / entropy / Betti curve

In [ ]:
perslayParameters["layer"]          = "Landscape"
#perslayParameters["layer"]          = "BettiCurve"
#perslayParameters["layer"]          = "Entropy"
perslayParameters["layer_train"]    = False
perslayParameters["lsample_num"]    = 3000
perslayParameters["lsample_init"]   = np.array(np.arange(-1.,2.,.001), dtype=np.float32)
perslayParameters["theta"]          = 100 # used only if layer is "BettiCurve" or "Entropy"
perslayParameters["final_model"]    = "identity"

In [ ]:
model = PerslayModel(name="perslay", diagdim=2, perslay_parameters=[perslayParameters], rho="identity")
vector = model([diagrams, empty_feats])

In [ ]:
#Plot representation
V = vector[0,:]
plt.figure()
if perslayParameters["perm_op"] == "topk":
    V = np.reshape(V, [-1, perslayParameters["keep"]])
    for k in range(perslayParameters["keep"]):
        plt.plot(V[:,k], linewidth=5.0)
else:
    plt.plot(V, linewidth=5.0)
plt.show()
    
# Plot weight
if perslayParameters["pweight"] == "grid":
    W = model.vars[0][0].numpy()
    weights = np.flip(W, 0)
    plt.figure()
    plt.imshow(weights, cmap="Purples", zorder=1)
    ((xm,xM),(ym,yM)) = perslayParameters["pweight_bnds"]
    [xs, ys] = perslayParameters["pweight_size"]
    plt.scatter([int(xs*(x-xm)/(xM-xm)) for x in diag[0][:,0]], 
                [ys-int(ys*(y-ym)/(yM-ym)) for y in diag[0][:,1]], 
                s=10, color="red", zorder=2)
    plt.show()
        
if perslayParameters["pweight"] == "gmix":
    means = model.vars[0][0][:2,:].numpy()
    varis = model.vars[0][0][2:,:].numpy()
    x = np.arange(-.5, 1.5, .001)
    y = np.arange(-.5, 1.5, .001)
    xx, yy = np.meshgrid(x, y)
    z = np.zeros(xx.shape)
    for idx_g in range(means.shape[1]):
        z += np.exp(-((xx-means[0,idx_g])**2 * (varis[0,idx_g])**2 
                    + (yy-means[1,idx_g])**2 * (varis[1,idx_g])**2 ))
    plt.contourf(xx, yy, z)
    plt.scatter(diag[0][:,0], diag[0][:,1], s=50, color="red")
    plt.show()